<a href="https://colab.research.google.com/github/zirid/pytorch/blob/master/GolfBallClassification_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 연결..
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive

In [ ]:
# Unzip data folder into TrainData folder 
# 압축해제
!unzip    /content/drive/MyDrive/Models.zip -d sample_data/TrainData

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#학습

from __future__ import print_function, division

# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import save
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy


DATA_INPUT_FOLDER =  "sample_data/TrainData/Models"
DATA_OUTPUT_FOLDER_PT = "sample_data/TrainData/Classification_Model.pt"
DATA_OUTPUT_FOLDER_ONNX = "sample_data/TrainData/Classification_Model2.onnx"


device = torch.device("cuda:0")

# Just normalization for validation
data_transforms = {
    
    'train': transforms.Compose([

        transforms.Resize((224, 224)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.RandomGrayscale(),
        transforms.RandomInvert(),
        transforms.RandomRotation(180),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'validation': transforms.Compose([

        transforms.Resize((224,224)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.RandomGrayscale(),
        transforms.RandomInvert(),
        transforms.RandomRotation(180),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)

                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


if __name__ ==  '__main__':

    data_dir = DATA_INPUT_FOLDER

    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),data_transforms[x]) for x in ['train', 'validation']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, shuffle=True, num_workers=2) for x in ['train', 'validation']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
    class_names = image_datasets['train'].classes

    # Get a batch of training data
    inputs, classes = next(iter(dataloaders['train']))

    # print(inputs.size())

    # Make a grid from batch
    out = torchvision.utils.make_grid(inputs)

    model_ft = models.resnet50(weights= models.ResNet50_Weights.DEFAULT)
    num_ftrs = model_ft.fc.in_features
    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    model_ft.fc = nn.Linear(num_ftrs, len(class_names)) # Class count !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size= 7, gamma=0.1)

    model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs= 8)

    device = torch.device("cuda:0")
    model_ft = model_ft.to(device)

    model_ft.eval()
    # An example input you would normally provide to your model's forward() method.
    x = torch.randn(1, 3, 224, 224 )

    x = x.to(device)
    torch_out = model_ft(x)
    print(torch_out)
    _, preds = torch.max(torch_out, 1)
    print('predicted: {}'.format(class_names[preds[0]]))

    # Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.

    traced_script_module = torch.jit.trace(model_ft, x)

    traced_script_module.save(DATA_OUTPUT_FOLDER_PT)

    # model export into ONNX format
    torch.onnx.export(
        model_ft,
        x,
        DATA_OUTPUT_FOLDER_ONNX,
        verbose= False,
        input_names=["input"],
        output_names=["output"],
        opset_version= 14
    )






FileNotFoundError: ignored

In [ ]:
#clear data  folder
#학습(TrainData)폴더 삭제 
import shutil
shutil.rmtree( "sample_data/TrainData", ignore_errors=True)
shutil.rmtree( "sample_data/Models.zip", ignore_errors=True)